# 파이썬 기반의 AI를 위한 기초수학, 확률및통계

In [1]:
## 강봉주 
## bonjour.kang@gmail.com
##
## 범주형 자료 분석
##

## 모비율의 추정 및 검정

### 데이터 구성

In [2]:
# 필요한 패키지
import numpy as np
from scipy import linalg as la
import matplotlib.pyplot as plt
import pandas as pd

import scipy.stats as ss
import statsmodels.formula.api as smf

pd.__version__

'0.25.3'

In [3]:
# [HFWS] 통계청 마이크로데이터  2018년 가계금융조사 데이터
url = 'https://github.com/bong-ju-kang/kmu-mba-statistics-winter/raw/master/data/MDIS_2018_HFWS.txt'
df = pd.read_csv(url, header=None)
df.head(2)

,0,1,2,3,4,5,6,7,8,9,...,151,152,153,154,155,156,157,158,159,160
0,2018,10200261,P,1127.905175,NaN,2255.0,G2,1,2,G1,...,NaN,NaN,1.0,80.0,60,150,4.0,NaN,NaN,NaN
1,2018,10200371,P,2223.797699,NaN,4515.0,G1,1,4,G2,...,NaN,NaN,1.0,70.0,150,200,4.0,NaN,NaN,NaN


In [4]:
df.shape

(18640, 161)

In [5]:
# 가구주 성별
gender = df.iloc[:, 7] 
gender.value_counts()

1    14061
2     4579
Name: 7, dtype: int64

In [6]:
# 가구원수
num_members = df.iloc[:, 8] 
np.round(num_members.mean())

3.0

In [7]:
# 교육 수준
edulevel = df.iloc[:, 14] 
edulevel.value_counts()
# 1 안 받음(미취학 포함)
# 2 초등학교
# 3 중학교
# 4 고등학교
# 5 대학(3년제 이하)
# 6 대학교(4년제 이상)
# 7 대학원 이상

4    5883
6    4427
2    2362
3    2033
5    1836
7    1068
1    1031
Name: 14, dtype: int64

In [8]:
# 가구주 만나이
hhage = df.iloc[:, 20] 
np.round(hhage.mean())

56.0

In [9]:
# 가구주 혼인상태
maritalstatus = df.iloc[:, 24] 
maritalstatus.value_counts()
# 1 미혼
# 2 배우자있음
# 3 사별
# 4 이혼

2    12429
3     2682
4     1913
1     1616
Name: 24, dtype: int64

In [10]:
# 입주형태
housetype = df.iloc[:, 26] 
housetype.value_counts()
# G1 자기집
# G2 전세
# G3 월세
# G4 기타

G1    11573
G2     2963
G3     2800
G4     1304
Name: 26, dtype: int64

In [11]:
# 부채 보유 여부
debtflag = df.iloc[:, 29] 
debtflag.value_counts()
# G1 부채 보유
# G2 부채 미보유

G1    11185
G2     7455
Name: 29, dtype: int64

In [12]:
# 자산
asset = df.iloc[:, 40] 
np.round(asset.mean(), 3)

40053.221

In [13]:
# 금융 자산
fasset = df.iloc[:, 41] 
np.round(fasset.mean(), 3)

9805.399

In [14]:
# 실물자산 
tasset = df.iloc[:, 46] 
np.round(tasset.mean(), 3)

30247.821

In [15]:
# 담보대출
mloan =  df.iloc[:, 56] 
np.round(mloan.mean(), 3)

3964.482

In [16]:
# 신용 대출
cloan = df.iloc[:, 80] 
np.round(cloan.mean(), 3)

688.78

In [17]:
# 순자산액
netasset = df.iloc[:, 108] 
np.round(netasset.mean(), 3)

33245.647

In [18]:
# 가구소득(경상소득)
income = df.iloc[:, 109] 
np.round(income.mean(), 3)

5363.971

In [19]:
# 지출
expense = df.iloc[:, 117] 
np.round(expense.mean(), 3)

2183.234

In [20]:
# 가구주 은퇴여부
retireflag = df.iloc[:, 153] 
retireflag.value_counts()

1.0    7412
2.0    1849
Name: 153, dtype: int64

### 모비율의 추정

In [21]:
# 모비율 확인
pratio = edulevel[edulevel > 4].shape[0] / len(edulevel)
np.round(pratio, 3)

0.393

In [22]:
# 표본 추출
np.random.seed(123)
index = np.random.choice(len(df), 100, replace=False)
sample = edulevel[index]

# 표본 비율
sratio = sample[sample > 4].shape[0] / len(sample)
np.round(sratio, 3)

0.38

In [23]:
# 신뢰 구간
# 유의 수준
alpha = 0.05

# 표본 크기
n = len(sample)

# 분위수
z_alpha_2 = ss.norm.ppf(1-alpha/2)

# 신뢰구간
lower = sratio - z_alpha_2 * np.sqrt(sratio*(1-sratio)/n)
upper = sratio + z_alpha_2 * np.sqrt(sratio*(1-sratio)/n)
np.round([lower, upper], 3)

array([0.285, 0.475])

In [24]:
# 표본이 작은 경우: 모비율 검정
# p-value 계산

# 표본 크기
n = len(sample)

# 모 비율
p = 0.39

# 표본 이벤트 건수
nphat = sample[sample > 4].shape[0]

# p-value 계산
pvalue = 2*(1 - ss.binom.cdf(nphat, n=n, p=p))
np.round(pvalue, 3)

1.076

In [25]:
# 표본이 큰 경우
# 검정 통계량
z0 = (nphat - n*p)/np.sqrt(n*p*(1-p))

# p-value 계산
pvalue = (1 - ss.norm.cdf(z0))*2
np.round(pvalue, 3)

1.162

## 모 비율의 비교

In [26]:
# 관심 대상 변수로 구성된 데이터 구성
df1 = pd.DataFrame({'debtflag':debtflag, 'retireflag':retireflag})

# 결측값 제거
df1.dropna(inplace=True)

# 결측값 제거 확인
df1.isna().sum()

debtflag      0
retireflag    0
dtype: int64

In [27]:
# 교차표 생성
freq = pd.crosstab(index=df1["retireflag"], columns=df1["debtflag"], margins=True)
freq.columns = ["Y","N", 'RowTotal']
freq.index= ["retire_Y","retire_N", "ColTotal"]
freq

,Y,N,RowTotal
retire_Y,4869,2543,7412
retire_N,643,1206,1849
ColTotal,5512,3749,9261


In [28]:
# 행 합에 대한 비율: 은퇴 여부에 따른 부채 보유 비율
sratio = np.round(freq['Y'] / freq['RowTotal'], 3)
sratio

retire_Y    0.657
retire_N    0.348
ColTotal    0.595
dtype: float64

In [29]:
# 은퇴 여부 그룹의 크기
n = freq['RowTotal']
n

retire_Y    7412
retire_N    1849
ColTotal    9261
Name: RowTotal, dtype: int64

In [30]:
# 표본비율 차
dratio = sratio[0] - sratio[1]
dratio.round(3)

0.309

In [31]:
# 표준오차 계산
se = np.sqrt(sratio[0]*(1-sratio[0])/n[0] + sratio[1]*(1-sratio[1])/n[1] )
se.round(5)

0.01237

In [32]:
# 정규 분위수 계산
alpha = 0.05
z_alpha_2 = ss.norm.ppf(1-alpha/2)
z_alpha_2.round(3)

1.96

In [33]:
# 신뢰 구간 계산
lower = dratio - z_alpha_2 * se
upper =  dratio + z_alpha_2 * se
np.round([lower, upper], 4)

array([0.2847, 0.3333])

In [34]:
# 모비율 차의 유의성 검정
# 검정 통계량
z0 = dratio / se
z0.round(3)

24.972

In [35]:
# p-value 계산
pvalue = (1 - ss.norm.cdf(z0))*2
pvalue

0.0

## 동질성 검증

In [36]:
# 데이터 구성
ins = np.array([[100, 900],
               [200, 4500]])
ins

array([[ 100,  900],
       [ 200, 4500]])

In [37]:
# 열별 합
colSum = ins.sum(axis=0)
colSum

array([ 300, 5400])

In [38]:
# 전체 건수에 대한 이탈과 비이탈 비율
attratio = colSum / ins.sum()
attratio.round(3)

array([0.053, 0.947])

In [39]:
# 행별 합
rowSum = ins.sum(axis=1)
rowSum

array([1000, 4700])

In [40]:
# 기대 빈도 계산
expected = rowSum.reshape(-1, 1) @ colSum.reshape(1, -1) / ins.sum()
expected.round(3)

array([[  52.632,  947.368],
       [ 247.368, 4452.632]])

In [41]:
# 검증 통계량 계산
ts = np.sum((ins-expected)**2 / expected)
ts.round(3)

54.574

In [42]:
# 동질성 검증
# 유의 수준 정의
alpha = 0.05

# 자유도 정의
df = (ins.shape[0]-1) * 1

# 카이제곱 분위수 계산
ss.chi2.ppf(1-alpha, df=df)

# p-value 계산
pvalue = 1 - ss.chi2.cdf(ts, df=df)
pvalue.round(3)

0.0

## 독립성 검증

In [43]:
# 데이터 구성
# 관심 대상 변수로 구성된 데이터 구성
df2 = pd.DataFrame({'maritalstatus':maritalstatus, 'housetype':housetype})

# 결측값 여부 확인
df2.isna().sum()

maritalstatus    0
housetype        0
dtype: int64

In [44]:
# 교차표 구성
freq = pd.crosstab(index=df2["maritalstatus"], columns=df2["housetype"], margins=False)
freq.columns = ["자기집","전세", '월세', '기타']
# G1 자기집
# G2 전세
# G3 월세
# G4 기타
freq.index= ["미혼","배우자있음", "사별", '이혼']
# 1 미혼
# 2 배우자있음
# 3 사별
# 4 이혼
freq

,자기집,전세,월세,기타
미혼,479,385,599,153
배우자있음,8930,1864,1066,569
사별,1529,317,393,443
이혼,635,397,742,139


In [45]:
#  카이제곱 검증
chi2, p, dof, expected = ss.chi2_contingency(freq.values, correction=False)

# 검증 통계량
chi2.round(3)

2911.535

In [46]:
# p-value
p

0.0

In [47]:
# 자유도
dof

9

In [48]:
# 기대 빈도
expected.round(0)

array([[1003.,  257.,  243.,  113.],
       [7717., 1976., 1867.,  869.],
       [1665.,  426.,  403.,  188.],
       [1188.,  304.,  287.,  134.]])

In [49]:
# 실제 빈도와의 차이
np.round(freq - expected, 0)

,자기집,전세,월세,기타
미혼,-524.0,128.0,356.0,40.0
배우자있음,1213.0,-112.0,-801.0,-300.0
사별,-136.0,-109.0,-10.0,255.0
이혼,-553.0,93.0,455.0,5.0


## 가변수 변환 또는 원 핫 인코딩(one hot encoding)

In [50]:
# 변수 정의
allvars = ['netasset', 'gender', 'num_members', 
           'edulevel', 'hhage', 'maritalstatus', 'housetype']

# 문자열에 해당하는 객체 가져오기
allseries = [eval(var) for var in allvars]

In [51]:
# 데이터 구성
sdf = pd.concat(allseries, axis=1)
sdf.columns = allvars
sdf.columns

Index(['netasset', 'gender', 'num_members', 'edulevel', 'hhage',
       'maritalstatus', 'housetype'],
      dtype='object')

In [52]:
# 기본적인 형태
print(pd.get_dummies(sdf[['netasset', 'maritalstatus']], columns=['maritalstatus']).head(3))

   netasset  maritalstatus_1  maritalstatus_2  maritalstatus_3  \
0         2                0                1                0   
1     58522                0                1                0   
2    101350                0                1                0   

   maritalstatus_4  
0                0  
1                0  
2                0  


In [53]:
# 범주값 1개 제거
print(pd.get_dummies(sdf[['netasset', 'maritalstatus']], columns=['maritalstatus'], 
                    drop_first=True).head(3))

   netasset  maritalstatus_2  maritalstatus_3  maritalstatus_4
0         2                1                0                0
1     58522                1                0                0
2    101350                1                0                0


In [54]:
# 모든 범주 변수의 가변수 변환
# 범주 변수 정의
cat_vars = ['gender', 'edulevel', 'maritalstatus', 'housetype']

# 가변수 변환
new_sdf = pd.get_dummies(sdf, columns=cat_vars, drop_first=True)
new_sdf.columns

Index(['netasset', 'num_members', 'hhage', 'gender_2', 'edulevel_2',
       'edulevel_3', 'edulevel_4', 'edulevel_5', 'edulevel_6', 'edulevel_7',
       'maritalstatus_2', 'maritalstatus_3', 'maritalstatus_4', 'housetype_G2',
       'housetype_G3', 'housetype_G4'],
      dtype='object')

In [55]:
# 선형회귀 적합
# 기호식 정의
fmla = 'netasset~' + '+'.join(list(set(new_sdf.columns) - {'netasset'}))
fmla

'netasset~maritalstatus_3+gender_2+housetype_G3+maritalstatus_2+housetype_G4+housetype_G2+edulevel_5+edulevel_3+edulevel_6+num_members+hhage+edulevel_7+edulevel_2+edulevel_4+maritalstatus_4'

In [56]:
# 선형회귀 적합
fit = smf.ols(fmla, data=new_sdf).fit()
print(fit.summary())

                            OLS Regression Results                            
Dep. Variable:               netasset   R-squared:                       0.177
Model:                            OLS   Adj. R-squared:                  0.177
Method:                 Least Squares   F-statistic:                     267.9
Date:                Wed, 11 Mar 2020   Prob (F-statistic):               0.00
Time:                        10:22:38   Log-Likelihood:            -2.2862e+05
No. Observations:               18640   AIC:                         4.573e+05
Df Residuals:                   18624   BIC:                         4.574e+05
Df Model:                          15                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept        -4.18e+04   3426.440    -